###  Created by Luis A. Sanchez-Perez (alejand@umich.edu).
<p><span style="color:green"><b>Copyright &#169;</b> Do not distribute or use without authorization from author.</span></p>

In [1]:
import pathlib
import pandas as pd
import re
import tensorflow as tf
import json

In [2]:
ONLY_NAME_REGEX = re.compile(r'([\w]*)')
BUFFER_SIZE = 1000

In [3]:
path = pathlib.Path('E:/datasets/regression/bodyfat.csv')
data = pd.read_csv(path)
data.columns = [re.match(ONLY_NAME_REGEX, column).group(1) for column in data.columns]
data.head()
labels = data.pop('Bodyfat')

In [4]:
ds = tf.data.Dataset.from_tensor_slices((dict(data), labels))
ds = ds.shuffle(BUFFER_SIZE)

In [5]:
entry, label = next(iter(ds))
for key, value in entry.items():
    entry[key] = tf.expand_dims(value, axis=0)

In [6]:
entry

{'Age': <tf.Tensor: shape=(1,), dtype=int64, numpy=array([42], dtype=int64)>,
 'Weight': <tf.Tensor: shape=(1,), dtype=float64, numpy=array([224.75])>,
 'Height': <tf.Tensor: shape=(1,), dtype=float64, numpy=array([74.75])>,
 'Neck': <tf.Tensor: shape=(1,), dtype=float64, numpy=array([38.5])>,
 'Chest': <tf.Tensor: shape=(1,), dtype=float64, numpy=array([106.7])>,
 'Abdomen': <tf.Tensor: shape=(1,), dtype=float64, numpy=array([105.7])>,
 'Hip': <tf.Tensor: shape=(1,), dtype=float64, numpy=array([111.8])>,
 'Thigh': <tf.Tensor: shape=(1,), dtype=float64, numpy=array([65.3])>,
 'Knee': <tf.Tensor: shape=(1,), dtype=float64, numpy=array([43.3])>,
 'Ankle': <tf.Tensor: shape=(1,), dtype=float64, numpy=array([26.])>,
 'Biceps': <tf.Tensor: shape=(1,), dtype=float64, numpy=array([33.7])>,
 'Forearm': <tf.Tensor: shape=(1,), dtype=float64, numpy=array([29.9])>,
 'Wrist': <tf.Tensor: shape=(1,), dtype=float64, numpy=array([18.5])>}

In [7]:
# Loads model locally to compare results from the cloud
model = tf.keras.models.load_model('saved')

In [8]:
model(entry)

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[25.839851]], dtype=float32)>

In [9]:
MODEL_NAME = 'body'
INPUT_DATA_FILE = 'instances.json'
VERSION_NAME = 'v1'
REGION = 'us-east1'

In [10]:
import os
os.environ['MODEL_NAME'] = MODEL_NAME
os.environ['INPUT_DATA_FILE'] = INPUT_DATA_FILE
os.environ['VERSION_NAME'] = VERSION_NAME
os.environ['REGION'] = REGION

In [11]:
observation = {key: tf.squeeze(tensor).numpy().astype(float) for key, tensor in entry.items()}
request = {'instances': [observation]}
with open(pathlib.Path(INPUT_DATA_FILE), 'w') as file:
    json.dump(request, file, indent=4)

In [12]:
request

{'instances': [{'Age': 42.0,
   'Weight': 224.75,
   'Height': 74.75,
   'Neck': 38.5,
   'Chest': 106.7,
   'Abdomen': 105.7,
   'Hip': 111.8,
   'Thigh': 65.3,
   'Knee': 43.3,
   'Ankle': 26.0,
   'Biceps': 33.7,
   'Forearm': 29.9,
   'Wrist': 18.5}]}

In [13]:
%%cmd
gcloud ai-platform predict --model=%MODEL_NAME% --version=%VERSION_NAME% --json-request=%INPUT_DATA_FILE%

Microsoft Windows [Version 10.0.18362.1139]
(c) 2019 Microsoft Corporation. All rights reserved.

(gcp) D:\Cloud Folders\Gloogle Drive\python\ai-examples\linear regression\keras gcp example>gcloud ai-platform predict --model=%MODEL_NAME% --version=%VERSION_NAME% --json-request=%INPUT_DATA_FILE%
DENSE_2
[25.83985710144043]

(gcp) D:\Cloud Folders\Gloogle Drive\python\ai-examples\linear regression\keras gcp example>

Using endpoint [https://ml.googleapis.com/]
